In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import uuid   # Unique identifier
import os
import time
import datetime
min_conf = .5

In [ ]:
#Loading my trained Port Detection Model. 

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp/weights/last.pt', force_reload=True)

In [ ]:
##Opening the USB Log File 
log_USB = open("USB_log.txt", 'a')

#Using OpenCV to capture feed as input
cap = cv2.VideoCapture(0)

# Initialize the state to 'UNKNOWN'
current_state = 'UNKNOWN'

while cap.isOpened():
    ret, frame = cap.read()
    
    # Make detections 
    results = model(frame)
    
    # Check if a USB is detected
    usb_detected = False
    for r in results.pred[0]:
        class_id = int(r[-1])
        if class_id == 16:
            usb_detected = True
            break
    
    # Update the current state based on the detection
    if usb_detected and current_state != 'USB':
        current_state = 'USB'
        current_time = datetime.datetime.now()
        time_string = current_time.strftime("%Y-%m-%d %H:%M:%S")
        print("USB is Detected")
        log_USB.write(time_string + " - USB is Detected!\n") 
        
    #If the state of the detected object changes to Ports_Open It will be logged
    elif not usb_detected and current_state != 'NO_USB':
        current_state = 'NO_USB'
        current_time = datetime.datetime.now()
        time_string = current_time.strftime("%Y-%m-%d %H:%M:%S")
        print("Ports are Open")
        log_USB.write(time_string + " - Ports are Open!\n") 

    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

log_USB.close() 
cap.release()
cv2.destroyAllWindows()
